# Feature Engineering & Target Definition

## Context & Previous Work
- In the previous version of the project, we used returns over a certain period of time to predict trends. This was clearly unsuccessful
- Then, we identified the Golden Cross rule to be the best way to classify financial trend over several different methods (in the target_testing notebook)
- We have with this target stable regimes with few transitions

## New Target: Transition Prediction

### Motivation
- We know that the Golden Cross rule is functionning to predict financial trend, but use it as a target to predict with ML models is useless, one could just compute it.
- What we will implement here is a "transition incoming" target. More explanations below

### Target Definition
- Target: "transition_incoming". in other words, the target will no longer be Bullish/Non-Bullish, but rather "a transition is coming/Nothing is coming" 
- Label = 1 during the **30-day window preceding** a transition
- Label = 0 otherwise
- This creates a prediction horizon: we want to detect transitions up to 30 days in advance

### Decision Rule (Rolling days confirmation)
- Identified issue: Classifying a transition with only one label 1 30 days before the transition happens is subject to false positive (which is very dangerous in finance)
- So, we implemented a rule: A transition is coming if during 10 days, at least 7 labels were equal to 1. It allows the prediciton to be more accurate and less strict than 10 out of 10 1 labels.
- Trade-off: With the 10-day rolling window, we effectively predict transitions with **20-30 days of advance notice** (depending on when the 7/10 threshold is met)

## Class Imbalance
- With 25 transitions over ~6100 trading days, and a 30-day labeling window, we expect approximately **12% positive labels**
- This class imbalance reflects market reality: regime transitions are rare events
- We will address this through:
  * Appropriate evaluation metrics (Precision, Recall, F1, AUC-PR)
  * Class weighting in models if needed
  * Focus on Precision over Recall (false positives are costly)

### Excluded Features (data leakage)

Since the financial regime is computed according to the Golden Cross rule, any features including directly or indirectly the usage of moving averages (MA, EMA, MACD...) are excluded to prevent data leakage

### Included Features
- Actual features : vol, returns, cumulative returns, RSI
- New features : ATR, Volume ROC, Stochastic oscillator

## Implementation Plan
1. Implement the new labeling function with 30-day window
2. Add new technical indicators (ATR, Volume ROC, Stochastic)
3. Verify no feature correlation/redundancy
4. Train baseline models with proper temporal split
5. Evaluate with Precision-focused metrics

In [3]:
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.data_loader import load_data


spy = load_data()
spy.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2000-01-03 00:00:00-05:00,93.388752,93.388752,90.632760,91.617043,8164300,0.0,0.0,0.0
1,2000-01-04 00:00:00-05:00,90.416220,90.750877,87.965356,88.034256,8089800,0.0,0.0,0.0
2,2000-01-05 00:00:00-05:00,88.152371,89.156339,86.459404,88.191742,12177900,0.0,0.0,0.0
3,2000-01-06 00:00:00-05:00,87.955539,89.136678,86.774399,86.774399,6227200,0.0,0.0,0.0
4,2000-01-07 00:00:00-05:00,88.388592,91.813896,88.231107,91.813896,8066500,0.0,0.0,0.0


In [5]:
from src.features import add_all_features, add_target_ma_cross